In [1]:
%load_ext Cython
import os
import pandas as pd
import anytree as at
import time

In [2]:
CWD=os.getcwd()
data_subdir='/data/'

columns=["id", "unix_ts", "ts", "key", "value"]
data_files=[CWD+data_subdir+f for f in os.listdir(CWD+data_subdir)]

In [3]:
df=pd.read_csv(data_files[1], sep=';', error_bad_lines=False, index_col=0, names=columns, header=None)
df

,unix_ts,ts,key,value
id,,,,
1,1,(invalid date),install,471292580
2,471287243,2010-12-15T10:49:30.970+0000,time|bootup,2010-12-15T10:49:30.970+0000
4,471287355,2010-12-15T10:49:31.082+0000,system|device,passion
5,471287356,2010-12-15T10:49:31.083+0000,system|apiversion,8
6,471287357,2010-12-15T10:49:31.084+0000,system|osbuildtype,user
7,471287364,2010-12-15T10:49:31.091+0000,system|osstring,2.2.1 FRG83D
8,471287368,2010-12-15T10:49:31.095+0000,system|swversion,0.9.6d
9,471287368,2010-12-15T10:49:31.095+0000,system|swbuild,416
10,471287369,2010-12-15T10:49:31.096+0000,system|freeinternal,63406080


In [12]:
%%cython --annotate

cdef class Node:
    cdef name
    cdef list children
    def __init__(self, name="", Node parent=None):
        self.name = name
        self.children=[]
        
        # No need for checks because already done in createNodesFromPath
        if parent is not None:
            parent.children.append(self)
        
cdef Node getChild(Node parent, child_name):
    cdef Node node
    for node in parent.children:
        if node.name == child_name:
            return node
    return None

# Adds a path of nodes (as a list of strings: ['top', 'mid', 'leaf']) to the root
# when they do not already exist in the root tree
cpdef Node createNodesFromPath(Node root, list path):
    cdef Node child, parent = root
    for node_name in path:
        child = getChild(parent, node_name)
        #print("'{}' children named '{}': {}".format(parent.name, node_name, child))
        if child is None:
            #print("'{}' has no child named '{}': creating it.".format(parent.name, node_name))
            parent=Node(node_name, parent=parent)
        else:
            #print("'{}' already has a child named '{}': going on.".format(parent.name, node_name))
            parent=child
    
    return root

In [13]:
percents_shown=20
root = Node("root")
N=df.shape[0]
t0=time.perf_counter()

for i, k in enumerate(df['key']):
    root=createNodesFromPath(root, k.split('|'))
    
    if i != 0 and (i-1) % (N/percents_shown) > i % (N/percents_shown):
        print("[{:.1f}s]{:.1f}% done.".format(time.perf_counter() - t0, i/N*100))
    
print("All done in {:.1f}s!".format(time.perf_counter() - t0))
print(at.RenderTree(root, style=at.AsciiStyle()).by_attr())

[1.9s]5.0% done.
[5.5s]10.0% done.
[10.0s]15.0% done.
[16.0s]20.0% done.


KeyboardInterrupt: 

In [21]:
txt=at.RenderTree(root, childiter=lambda iter: sorted(iter, key=lambda node: node.name), style=at.ContRoundStyle()).by_attr()

with open("../tree_structure.txt", "w") as f:
    f.write(txt)

In [63]:

def renderNode(node, depth, siblingsToDraw):
    for i, child in enumerate(node.children):
        if 

KeyboardInterrupt: 